# Emparelhamentos de delimitadores
-------------------------------------------------------------

>**Autor** Joel Paula - nº 93393
>
>**Professor** Luis Ramada Pereira
>
>Estrutura de Dados e Algoritmos
>
>LCD-PL | Ano letivo 2019/2020
>
>28-03-2020

## Enquadramento

No âmbito da disciplina de Estrutura de dados e algoritmos, o 1º trabalho pede-nos para criarmos um programa, na linguagem *Python*, capaz de verificar o correto (ou incorreto) emparelhamento de delimitadores numa expressão ou ficheiro de *Python*.

Para simplificar o problema, não são consideradas as situações de instruções/expressões multilinha (cujos delimitadores iniciem numa linha e terminem noutra), nem instruções dentro de uma *string*, de uma *docstring* ou de um comentário.

«O programa deve indicar na consola a expressão inserida pelo utilizador, acrescida do sufixo *“True”* ou *“False”*, consoante a expressão esteja bem ou mal formada, respetivamente.» (Enunciado do Trabalho 1 de Estrutura de dados e Algoritmos, 2020) [1]

Na lista de delimitadores do _Python_ (*Python Software Foundation*, 2020) [2] apenas 3 tipos precisam de emparelhamento: os parêntesis curvos e os retos, bem como as chavetas.

Por exemplo, caso o utilizador insira a expressão ``lista = [1, 2]``, será considerada bem emparelhada e, portanto, o resultado na consola deverá ser ``lista = [1, 2] True``.
Já a expressão ``lista = [1, 2)``, será considerada mal emparelhada, por não "fechar" devidamente o primeiro parêntesis reto, e, portanto, o resultado na consola deverá ser  ``lista = [1, 2) False``.

Estes exemplos simples são imediatos e não parecem exigir um algoritmo complicado para a sua resolução. É necessário pensar em expressões mais complexas para compreender a dimensão do problema. Por exemplo ``points = [((1, 2), 'red'), ((3, 4), 'green')]`` combina 2 tipos de delimitadores em 3 níveis de "profundidade".  
O exemplo indicado no próprio enunciado do trabalho apresenta os 3 tipos de delimitadores, com 4 níveis: ``lista = [(2, 3), {4, 5}, [6, 7], np.array([(2 + 3) * 5])]``. Para este tipo de expressões, nota-se a necessidade de uma estrutura mais complexa, que nos permita emparelhar vários níveis. E se intuitivamente nos possa parecer simples criar um algoritmo de emparelhamento multinível, quando mergulhamos nos detalhes do problema, a solução pode não ser trivial.

## Estrutura da Solução

A solução mais intuitiva é a utilização de uma pilha. Uma pilha é uma estrutura abstrata de dados que é uma coleção do tipo LIFO - Last In First Out (último elemento a entrar é o primeiro a sair).  Isso adapta-se perfeitamente á análise de expressões e emparelhamento de delimitadores de aberto com delimitadores de fecho - ao percorrer os caracteres da expressão, colocamos na pilha cada delimitador aberto e removemos da pilha quando encontramos o delimitador de fecho correspondente. Ao adicionar à pilha estamos a abrir os vários níveis e ao retiramos estamos a fechá-los. No final da expressão verificamos se a pilha está vazia - o que indicaria que todos os delimitadores abertos foram fechados - ou se ainda contém algum elemento - o que indicaria que algum delimitador aberto não foi devidamente emparelhado com um de fecho.

Essa é mesmo a solução apontada na bibliografia aconselhada no enunciado do trabalho (Miller & Ranum, 2013) [3].

![Imagem da pilha a emparelhar parêntesis e a falhar quando um parêntesis não é emparelhado!](Stack.gif)

Para este trabalho criamos uma implementação de uma pilha, com base numa lista (solução também sugerida na bibliografia já citada, no capítulo "*3.4 The Stack Abstract Data Type*" ).  
Com base nessa pilha, criámos um emparelhador de delimitadores capaz de percorrer todos os caracteres de uma expressão e indicar se todos os delimitadores estão ou não devidamente emparelhados. O algoritmo usado é baseado num similar apresentado na mesma fonte bibliográfica, no capítulo "*3.4.4 Balanced Symbols (A General Case)*", não sendo no entanto uma cópia fiel do mesmo (tal não seria sequer possível, tendo em conta que esse algoritmo apenas considera expressões que se componham exclusivamente dos símbolos a emparelhar).  
Adicionalmente, criámos uma forma simples de abrir e percorrer todas as linhas de um ficheiro de código fonte *Python*.

Finalmente, contruímos uma interface com o utilizador por linha de comando.


## Solução e testes

### Implementação de uma Pilha
Começamos por implementar uma pilha com base numa lista:

In [2]:
class Stack:
    def __init__(self):
        self.__stack = []

    def push(self, item):
        self.__stack.append(item)

    def pop(self):
        return self.__stack.pop()

    def peek(self):
        return self.__stack[len(self.__stack) - 1]

    def is_empty(self):
        return len(self.__stack) == 0

Tendo em conta os requisitos, uma lista dinâmica não parece ser um problema, uma vez que as operações de adicionar ou remover itens da mesma não serão muito "pesados", nem em termos de tempo, nem de memória - a ordem complexidade de ``list.append()`` é O(1) e de ``list.pop()`` também é O(1).  
Na verdade, todas as operações desta pilha têm complexidade O(1).  
O método ``list.append()`` adiciona um novo elemento no final da lista e o método ``list.pop()`` devolve e remove o último elemento da lista, o que se adapta perfeitamente ao funcionamento da lista. O topo da lista é sempre o último elemento da mesma, daí que o método ``list.peek()`` use o tamanho da lista para chegar ao índice do último elemento da lista.   
Nota: optamos por não implementar a operação ``size()``, uma vez que não teria utilidade no nosso programa, mas a sua implementação seria trivial - ``return len(self.__stack)``.

Para testar a pilha implementámos uma série de testes unitários:

In [4]:
# test_push
s = Stack()
s.push("test")
assert s.peek() == "test", "«push» not working"

# test_pop
s = Stack()
s.push("test")
s.push(7)
assert s.pop() == 7, "«pop» not showing latest pushed item"

# test_peek
s = Stack()
s.push(7.7)
assert s.peek() == 7.7, "«peek» not looking at latest pushed item"

# test_is_empty
s = Stack()
assert s.is_empty(), "stack should be empty on init"

# test_is_empty_after_push_and_pop
s = Stack()
s.push("test")
s.pop()
assert s.is_empty(), "stack should be empty after push and pop"


Espera-se que os testes não devolvam qualquer erro. Sendo que um erro indicará um problema na implementação da classe.

### Implementação de um emparelhador genérico

Para implementação do emparelhador optamos por criar uma classe com métodos estáticos, pois a classe não precisa de manter qualquer estado.


In [5]:
class ParenthesesChecker:
    opening_par = "{[("
    closing_par = "}])"

    @staticmethod
    def is_match(opening, closing):
        return ParenthesesChecker.opening_par.index(opening) == \
               ParenthesesChecker.closing_par.index(closing)

    @staticmethod
    def check(expression):
        s = Stack()
        result = True
        for char in expression:
            if char in ParenthesesChecker.opening_par:
                s.push(char)
            elif char in ParenthesesChecker.closing_par:
                if s.is_empty() or not ParenthesesChecker.is_match(s.pop(), char):
                    result = False
                    # Return already and spare us cycling through the whole string
                    break

        result = result and s.is_empty()
        return result

    @staticmethod
    def check_and_print(expression):
        result = ParenthesesChecker.check(expression)
        if result:
            print(expression.rstrip(), "\033[92m {}\033[00m".format("True"))
        else:
            print(expression.rstrip(), "\033[31m {}\033[00m".format("False"))
        return result

    @staticmethod
    def check_file(filename, print_results=False):
        result = True
        with open(filename) as fp:
            line = fp.readline()
            while line:
                if print_results:
                    result = result and ParenthesesChecker.check_and_print(line)
                else:
                    result = result and ParenthesesChecker.check(line)
                line = fp.readline()
        return result


- A implementação do verificador de emparelhamento centra-se no seu método ``check``. Este método é responsável por percorrer todos os caracteres da expressão que lhe é entregue e, usando uma pilha, verificar o correto emparelhamento dos delimitadores de abertura e fecho.  
 - Ao encontrar um delimitador de abertura, insere-o na pilha. 
 - Ao encontrar um delimitador de fecho, retira-o da pilha.  

 Exemplo 1: se a expressão for ``()``, o primeiro caractere é reconhecido como um delimitador de abertura e é colocado na pilha; o segundo é reconhecido como um delimitador de fecho e isso implica retirar um elemento da pilha. O resultado é ``True`` uma vez que não sobra nenhum elemento na pilha.  
 Exemplo 2: se a expressão for ``(()``, os dois caracteres iniciais são reconhecidos como delimitadores de abertura e são colocados na pilha; o terceiro é reconhecido como um delimitador de fecho e isso implica retirar um elemento da pilha. Neste caso o resultado seria ``False``, uma vez que sobra um elemento delimitador na pilha, por fechar.  

 - Se o delimitador de fecho não corresponder ao delimitador de abertura na pilha (por exemplo, no caso ``( ]``), então o algoritmo marca a expressão como não devidamente emparelhada imediatamente.  

 Optámos por terminar imediatamente a avaliação da expressão assim que é detetado um desemparelhamento, para poupar ciclos de trabalho improdutivos, mas basta remover a instrução ``break`` dentro do ciclo ``for`` para que o mesmo percorra toda a expressão.  
 O algoritmo despreza todos os caracteres que não correspondam a delimitadores. Seria interessante adicionar suporte para a deteção de blocos de comentários, ou *strings*, extendendo assim a utilização do algoritmo.


- O método ``match`` apenas verifica se o delimitador de abertura corresponde ao delimitador de fecho, usando uma simples correspondência do respetivo caractere a um índice na "lista" de delimitadores definida inicialmente na classe.


- O método ``check_and_print`` estende o método ``check``, colocando na consola a expressão seguida do resultado da avaliação do emparelhamento - o sufixo ``True`` se existe um correto emparelhamento de delimitadores e o ``False`` se não. Adicionalmente, usa as cores verde ou vermelho para colorir o sufixo, consoante o caso.


- Finalmente, o método ``check_file`` permite verificar um ficheiro completo, linha por linha. Para isso, abre o ficheiro e percorre cada linha, fazendo a verificação da mesma com o método ``check``. Usando o parâmetro opcional ``print_results``, escolhe se a verificação imprime ou não imprime os resultados na consola, ao selecionar se usa realmente o método ``check``, ou o ``check_and_print``. Por omissão, não se imprime nada na consola.


### Métodos de teste do emparelhador de genérico

Alguns métodos para teste de emparelhamento de expressões, com vários casos de expressões bem e mal emparelhadas:

In [6]:
ParenthesesChecker.check_and_print("()")
ParenthesesChecker.check_and_print(")")
ParenthesesChecker.check_and_print("(]")
ParenthesesChecker.check_and_print("(()")
ParenthesesChecker.check_and_print("({}[])")
ParenthesesChecker.check_and_print("([{}])")
ParenthesesChecker.check_and_print("lista = (2,3), {4,5}, [6,7], np.array([(2+3)*5])")
ParenthesesChecker.check_and_print("lista = [(2,3), {4,5}, [6,7], np.array([(2+3)*5])")
ParenthesesChecker.check_and_print("lista = [(2,3), {4,5}, [6,7], np.array([(2+3)*5])]")

()  True
)  False
(]  False
(()  False
({}[])  True
([{}])  True
lista = (2,3), {4,5}, [6,7], np.array([(2+3)*5])  True
lista = [(2,3), {4,5}, [6,7], np.array([(2+3)*5])  False
lista = [(2,3), {4,5}, [6,7], np.array([(2+3)*5])]  True


True

Resultados esperados:
```
()  True
)  False
(]  False
(()  False
({}[])  True
([{}])  True
lista = (2,3), {4,5}, [6,7], np.array([(2+3)*5])  True
lista = [(2,3), {4,5}, [6,7], np.array([(2+3)*5])  False
lista = [(2,3), {4,5}, [6,7], np.array([(2+3)*5])]  True
```


Para testar o processamento de ficheiros, temos dois ficheiros na pasta da aplicação, um que não apresenta problemas de emparelhamento de delimitadores ``TestOk.py``, e outro com problemas``TestNok.py``.

Utilizando esses ficheiros para testar o método ``check_file``:

In [7]:
print("##################################### TestOk.py")
result = ParenthesesChecker.check_file("TestOk.py", True)
print("")
print("##################################### TestNok.py")
result = ParenthesesChecker.check_file("TestNok.py", True)

##################################### TestOk.py
# Python program to print  True
# colored text and background  True
def prRed(skk): print("\033[91m] {}\033[00m]" .format(skk))  True
def prGreen(skk): print("\033[92m] {}\033[00m]" .format(skk))  True
def prYellow(skk): print("\033[93m] {}\033[00m]" .format(skk))  True
def prLightPurple(skk): print("\033[94m] {}\033[00m]" .format(skk))  True
def prPurple(skk): print("\033[95m] {}\033[00m]" .format(skk))  True
def prCyan(skk): print("\033[96m] {}\033[00m]" .format(skk))  True
def prLightGray(skk): print("\033[97m] {}\033[00m]" .format(skk))  True
def prBlack(skk): print("\033[98m] {}\033[00m]" .format(skk))  True
  True
  True
prCyan("Hello World, ")  True
prYellow("It's")  True
prGreen("Geeks")  True
prRed("For")  True
prGreen("Geeks")  True
  True
setMenu = {"Eggs", "Bacon"}  True
print(setMenu)  True
# > {'Bacon', 'Eggs'}  True
setMenu = set(list(setMenu) + list({"Spam"}))  True
print(setMenu)  True
# > {'Bacon', 'Spam', 'Eggs'}  True


### Interface de linha de comando com utilizador

Finalmente, a última parte do programa implementa uma interface de linha de comando com o utilizador, que permite ao utilizador indicar um ficheiro ou uma expressão para avaliação:

In [11]:
def process_expression():
    result = input("por favor, insira uma expressão para avaliação:")
    if result: ParenthesesChecker.check_and_print(result)
    return result


def process_file():
    file_path = input("Por favor, indique o caminho para o ficheiro a avaliar:")
    file_path = file_path.strip()
    if file_path:
        ParenthesesChecker.check_file(file_path, True)
    return file_path


response = "?"
while response:
    response = input("Emparelhamento de delimitadores\nO que deseja avaliar? Uma (E)xpressão ou um (F)icheiro ?")
    if response == "E":
        response = process_expression()
    elif response == "F":
        response = process_file()


import unittest  True
import paramParser  True
  True
  True
class Tests(unittest.TestCase):  True
  True
    def testOneIntValue(self):  True
        result = paramParser.main(["-d C:\\Path\\to\\yourfile\\xxx.m", "-p 'xxxParamName'", "-v 3"])  True
        self.assertTrue('xxxParamName.Value= 3', result)  True
  True
    def testMultipleIntValues(self):  True
        result = paramParser.main(["-d C:\\Path\\to\\yourfile\\xxx.m", "-p 'xxxParamName'", "-v 3"])  True
        self.assertTrue('[ xxxParamName.Value = [ 1 2 3 ]', result)  False


Esta interface permite ao utilizador selecionar se pretende avaliar uma expressão inserindo a letra "E", ou um ficheiro inteiro inserindo a letra "F".

Para avaliar uma expressão a interface pede ao utilizador para inserir a expressão, que é imediatamente avaliada, apresentando a expressão e o resultado na concola.

Para o caso de um ficheiro, é pedido o caminho para o ficheiro. O utilizador pode inserir apenas o nome de um ficheiro que exista na mesma pasta onde corre o programa, um caminho relativo à mesma ou um caminho absoluto desde a raiz do sistema.  
Esse ficheiro será avaliado linha por linha e cada linha será apresentada na consola, em conjunto com o resultado da avaliação.

Para sair do ciclo e do programa, basta ao utilizador inserir uma resposta vazia.

## Conclusões

Este trabalho permitiu-nos exercitar a criação e utilização de uma estrutura de Pilha e um algoritmo que a usa para emparelhar delimitadores e validar expressões. A Pilha é uma estrutura fundamental em ciência da computação, por exemplo usada para guardar o estado de navegação numa aplicação e permitir ao utilizador usar o tão habitual botão "back/anterior". 


A ordem de complexidade das operações da Pilha é O(1), no entanto o emparelhamento de delimitadores obriga-nos a percorrer todos os caracteres da expressão a analisar, pelo que a sua ordem de complexidade é O(n).

Não seria difícil estender o nosso algoritmo para permitir-lhe fazer emparelhamento de delimitadores de *strings* (deixando de analisar o que está dentro das mesmas) e reconhecer blocos de comentários. Mesmo o processamento de instruções multilinha poderia ser feito, analisando as linhas todas como se fizessem parte da mesma expressão - partilhando a mesma pilha.


## Referências bibliográficas

[1]: Enunciado do Trabalho 1 de Estrutura de dados e Algoritmos. (8 de Março de 2020). Obtido em 03 28, 2020, de https://e-learning.iscte-iul.pt/bbcswebdav/pid-362232-dt-content-rid-880437_1/courses/2019_03587-2_0324/Trabalho-1-C.pdf

[2]: Python Software Foundation. (28 de Março de 2020). 2. Lexical analysis. Obtido de The Python Language Reference: https://docs.python.org/3/reference/lexical_analysis.html#delimiters

[3]: Miller, B., & Ranum, D. (2013). Problem Solving with Algorithms and Data Structures using Python - Release 3.0.
